# Анализ пользовательского взаимодействия с карточками статей в Яндекс.Дзен

## Задача:
разработка дашборда для менеджеров по анализу контента.

## Состав данных для дашборда:

История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);

Разбивка событий по темам источников;

Таблица соответствия тем источников темам карточек;

По каким параметрам данные должны группироваться:

    дата и время;
    
    тема карточки;
    
    тема источника;
    
    возрастная группа;

Характер данных:

    история событий по темам карточек — абсолютные величины с разбивкой по минутам;

    разбивка событий по темам источников — относительные величины (% событий);
    
    соответствия тем источников темам карточек - абсолютные величины;

Важность: все графики имеют равную важность

После обработки сырых данных в нашем распоряжении есть таблица dash_visits за период с 18:28 24.09.2019 по 19:00 24.09.2019 со следующими данными:

record_id - идентификационный номер карточки;

item_topic - тема карточки;

source_topic - тема источника;

age_segment - возрастная категория пользователя;

dt - дата посещения;

visits - количество посещений/просмотров карточки.

## Ссылка на дашборд
https://public.tableau.com/app/profile/ekaterina.zatirka/viz/Project_Yandex_Zen/Yandex_Zen?publish=yes

## Ссылка на презентацию
https://drive.google.com/file/d/1f6lh3XMh1t71EkoW6OjilVTjqg9WvRvJ/view?usp=sharing

## Выгрузка данных, предварительное изучение.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
!pip install psycopg2-binary 

In [2]:
db_config = {'user': '', # имя пользователя
            'pwd': '', # пароль
            'host': '',
            'port': , # порт подключения
            'db': ''} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

query = '''SELECT *
            FROM dash_visits'''

dash_visits = pd.io.sql.read_sql(query, con = engine) 

In [3]:
display(dash_visits.head(10))
print(dash_visits.info())

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27
5,1040602,Деньги,Авто,18-25,2019-09-24 18:57:00,22
6,1040603,Деньги,Авто,18-25,2019-09-24 18:58:00,24
7,1040604,Деньги,Авто,18-25,2019-09-24 18:59:00,20
8,1040605,Деньги,Авто,18-25,2019-09-24 19:00:00,5
9,1040606,Деньги,Авто,26-30,2019-09-24 18:29:00,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30745 entries, 0 to 30744
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   record_id     30745 non-null  int64         
 1   item_topic    30745 non-null  object        
 2   source_topic  30745 non-null  object        
 3   age_segment   30745 non-null  object        
 4   dt            30745 non-null  datetime64[ns]
 5   visits        30745 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 1.4+ MB
None


Пропусков в полученной таблице нет, типы данных корректны.

In [4]:
print('Время первого события в таблице', dash_visits['dt'].min())
print('Время последнего события в таблице',dash_visits['dt'].max())

Время первого события в таблице 2019-09-24 18:28:00
Время последнего события в таблице 2019-09-24 19:00:00


In [5]:
print('Дубликатов в полученной таблице', dash_visits.duplicated().sum())

Дубликатов в полученной таблице 0


In [8]:
print('В рассматриваемом периоде совершено:', dash_visits['visits'].sum(), 'визитов')

В рассматриваемом периоде совершено: 310207 визитов


In [6]:
print('Темы карточек:', dash_visits['item_topic'].unique())
print()
print('Количество тем карточек:', dash_visits['item_topic'].nunique())
print()
print('Темы источников:',dash_visits['source_topic'].unique())
print()
print('Количество тем источников:', dash_visits['source_topic'].nunique())
print()
print('Возрастные категории пользователей:',dash_visits['age_segment'].unique())

Темы карточек: ['Деньги' 'Дети' 'Женская психология' 'Женщины' 'Здоровье' 'Знаменитости'
 'Интересные факты' 'Искусство' 'История' 'Красота' 'Культура' 'Наука'
 'Общество' 'Отношения' 'Подборки' 'Полезные советы' 'Психология'
 'Путешествия' 'Рассказы' 'Россия' 'Семья' 'Скандалы' 'Туризм' 'Шоу'
 'Юмор']

Количество тем карточек: 25

Темы источников: ['Авто' 'Деньги' 'Дети' 'Еда' 'Здоровье' 'Знаменитости' 'Интерьеры'
 'Искусство' 'История' 'Кино' 'Музыка' 'Одежда' 'Полезные советы'
 'Политика' 'Психология' 'Путешествия' 'Ремонт' 'Россия' 'Сад и дача'
 'Сделай сам' 'Семейные отношения' 'Семья' 'Спорт' 'Строительство'
 'Технологии' 'Финансы']

Количество тем источников: 26

Возрастные категории пользователей: ['18-25' '26-30' '31-35' '36-40' '41-45' '45+']


In [7]:
dash_visits.to_csv(r'C:\Users\Ekaterina\Documents\dash_visits.csv')

Выгружаю данные в файл csv для дальнейшего построения дашборда в Tableau Public.